In [ ]:
from IPython.display import display, Markdown, clear_output, HTML, Javascript
import ipywidgets as ipw
import json
import subprocess
import atexit
import os
import shutil
from ai_agent import ai_agent
from src import widgets, utils
import markdown

In [ ]:
CONFIG = utils.read_json("schema/config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis_aiida()
GOOGLE_API_KEY = utils.read_json("/home/jovyan/gemini_api.json")

openbis_agent = ai_agent.OpenBISAgent(
    google_api_key=GOOGLE_API_KEY["api_key"]
)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key '$defs' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key '$defs' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key '$defs' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key '$defs' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [ ]:
increase_buttons_size = HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))

prompt_label = ipw.HTML(value = "<span style='font-weight: bold; font-size: 12px;'>Prompt:</span>")

prompt_textarea = ipw.Textarea(
    layout = ipw.Layout(width = '800px', height = '100px'),
    style = {"description_width": "110px"}
)

enter_button = ipw.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Enter', icon = 'arrow-right', layout = ipw.Layout(width = '100px', height = '50px')
)

prompt_hbox = ipw.HBox(
    children = [
        prompt_label,
        prompt_textarea,
        enter_button
    ]
)

chat_label = ipw.HTML(value = "<span style='font-weight: bold; font-size: 12px;'>Chat:</span>")
chat_html = ipw.HTML(value = "<div style='border: 1px solid grey; padding: 10px; margin: 10px;'> <p><b>Agent:</b> Hi! I am openBIS chatbot and I am here to answer your questions about openBIS. How may I help you today?")
chat_hbox = ipw.HBox(children = [chat_label, chat_html])

quit_button = ipw.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)

In [ ]:
def close_notebook(b):
    display(Javascript(data = 'window.location.replace("home.ipynb")'))
    
def ask_chatbot(change):
    prompt = prompt_textarea.value
    chat_html.value = chat_html.value + f"<p><b>User:</b> {prompt}</p>"
    try:
        agent_response = openbis_agent.ask_question(prompt)
        response_text = agent_response[-1].content
    except Exception as e:
        response_text = "Internal Server Error. Repeat your prompt."
    chat_html.value = chat_html.value + "<p><b>Agent:</b> " + markdown.markdown(response_text) + "</p>"

# openBIS chatbot

#### Note: This agent uses model Gemini Flash 2.5. The answers may contain some errors/hallucinations.

In [ ]:
display(increase_buttons_size)
display(chat_hbox)
display(prompt_hbox)
display(quit_button)
enter_button.on_click(ask_chatbot)
quit_button.on_click(close_notebook)

In [ ]:
# x = openbis_agent.ask_question("Which experiments are available? Give me a summary of them")
# x = openbis_agent.ask_question("Is there an experiment that uses molecule 704a? If yes, give me a summary of it including the name of sample that was used and the sample that was produced by each process step and a small description of the actions and observables.")
# x = openbis_agent.ask_question("Give me a substance with smiles CCO")
# x = openbis_agent.ask_question("Is there a simulation in openBIS? If yes give me a summary")
# x = openbis_agent.ask_question("Considering the geometry optimisation, what was the used molecule?")
# x = openbis_agent.ask_question("Find me all the 2d materials in openBIS")
# x = openbis_agent.ask_question("Find me objects created between 17th July 2025 and 18th July 2025")
# x = openbis_agent.ask_question("Is instrument CreaTec THz-STM in openBIS? If yes, how many pumps does it have?")
# x = openbis_agent.ask_question("What object types exist in openBIS?")
# x = openbis_agent.ask_question("Yes. No filters give me all that you find")
# x = openbis_agent.ask_question("give me all the atomistic models")
# x

[HumanMessage(content='give me all the atomistic models', additional_kwargs={}, response_metadata={}, id='ad5ee296-e41b-4ca0-932d-8cef81b69e17'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_openbis_objects', 'arguments': '{"type": "ATOMISTIC_MODEL"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--aa4fd2a8-5abb-444d-9c69-4a22d35b990a-0', tool_calls=[{'name': 'get_openbis_objects', 'args': {'type': 'ATOMISTIC_MODEL'}, 'id': '5afa55e8-1d0d-4302-b38d-2320a707a40a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 7682, 'output_tokens': 21, 'total_tokens': 7703, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='[{"permId": "20250804112117202-3633", "type": "ATOMISTIC_MODEL", "properties": {"name": "C14H4", "description": null, "comments": null, "atomistic_model.wfms_uuid": "14ca2669-0846-4038-abb9-de258ec62edd", "atomistic_model.cell": "{\\"cell\\"

In [5]:
# from ai_agent.tools import inventory_tools
# from pydantic_schema import openbis_objects

# x = openbis_objects.Substance(
#     # empa_number = 650,
#     # batch = "a",
#     molecules = [
#         openbis_objects.Molecule(
#             smiles = "CCO"
#         )
#     ]
# )

# inventory_tools.get_substances_by_attributes(x)

# from pydantic_schema import openbis_objects
# openbis_objects.Substance(
#     name = "TUD-156", 
#     empa_number = 704,
#     # object_status = "Active"
# ).model_dump_json()

In [ ]:
# for state in openbis_agent.graph.get_state_history(openbis_agent.agent_config):
#     print(state.values["messages"])

[HumanMessage(content='Find me substances 670a and 704a', additional_kwargs={}, response_metadata={}, id='30a98aa5-c20d-4563-8fc1-9e9b0d8b7108'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_substances_by_attributes', 'arguments': '{"substance": {"batch": "a", "empa_number": 704.0}}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'LOW', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run--35ed375d-34c9-4214-87cf-0240f8ae963c-0', tool_calls=[{'name': 'get_substances_by_attributes', 'args': {'substance': {'batch': 'a', 'empa_number': 670.0}}, 'id': '6d4d1c21-d2be-47b5-b1f8

In [ ]:
# openbis_agent.get_graph_replay()

Num Messages:  4 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  1 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  0 Next:  ('__start__',)
--------------------------------------------------------------------------------


In [ ]:
# exp = OPENBIS_SESSION.get_experiment("20250717115224276-3595")
# exp.get_objects()[0].registrationDate

In [ ]:
# all_messages = []
# for state in openbis_agent.graph.get_state_history(openbis_agent.agent_config):
#     all_messages.extend(state.values["messages"])
# all_messages

In [ ]:
# obj_type = OPENBIS_SESSION.get_object_types()[10]
# properties_df = obj_type.get_property_assignments().df
# properties_codes = properties_df["code"]
# properties_datatypes = properties_df["dataType"]
# properties_info = []
# for code, datatype in zip(properties_codes, properties_datatypes):
#     property_info = {
#         "code": code.lower(),
#         "datatype": datatype.lower()
#     }
#     properties_info.append(property_info)

# properties_info